In [1]:
import datetime
import time

In [2]:
import logging
import os
import sys
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)
# from pyspark.sql import Row
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.6-src.zip'))

# Initialize PySpark to predefine the SparkContext variable 'sc'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0
      /_/

Using Python version 3.5.3 (default, Nov 23 2017 11:34:05)
SparkSession available as 'spark'.


sekarang cek ada gak pyspark, sc dan spark nya
*note : klo kodenya error karena hive initiate atau metastore_db, kamu tinggal hapus folder metastore_db yang ada di folder sama kayak ipython ini

In [3]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [4]:
spark

In [5]:
import pyspark

In [ ]:
# sc.setLogLevel("INFO")

In [6]:
sc._conf.getAll()

[('spark.app.id', 'local-1521342804411'),
 ('spark.executor.cores', '9'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.memory', '15g'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.driver.host', 'vmi130541.contaboserver.net'),
 ('spark.driver.port', '38867'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.cores', '1')]

harusnya udah ada smua kan..., nah skarang untuk pemahaman yang lebih jos...
1. setelah kamu bikin sc, dan spark session, kamu punya sparkUI, lokasi sparkUI nya ada klo kamu jalanin kode ini
2. kamu bisa check UI itu ketika kamu submit job ke spark
3. kamu juga bisa cek log di tmux jendela sebelah kanan ```tmux a -t jup``` . klo mau scroll window tmux tinggal ctrl+b -> [ trus kamu pake arrow up and down buat scroll nya, klo mau liat log atasnya

In [7]:
sc._jsc.sc().uiWebUrl().get()

'http://vmi130541.contaboserver.net:4042'

In [ ]:
from pyspark.sql.functions import input_file_name
from pyspark.sql.functions import monotonically_increasing_id

def convert_float(x):
    new_x = {}
    try:
        new_x['Close'] = float(x['Close'])
        new_x['High'] = float(x['High'])
        new_x['Low'] = float(x['Low'])
        new_x['Open'] = float(x['Open'])
    except:
        new_x['Close'] = float(x['Close'])
        new_x['High'] = float(x['Close'])
        new_x['Low'] = float(x['Close'])
        new_x['Open'] = float(x['Close'])
    
    new_x['Date'] = x['Date']
    new_x['Volume'] = float(x['Volume'])
    new_x['StockCode'] = x['filename'].split('/')[5][:4]
    return new_x
    
days_to_be_stacked = 30

In [ ]:
df = spark.read.format("csv").option("header", "true").load("until31_2017/*.csv")
df = df.withColumn("filename",input_file_name())
# df = df.withColumn("id", monotonically_increasing_id())
data_rdd = df.rdd.map(convert_float)

In [ ]:
df_2 = data_rdd.map(lambda x: ( (x['StockCode'], x) ))  \
                .groupByKey().mapValues(list)

In [60]:
df_2.take(1)

[('smmt',
  [{'Close': 133.0,
    'Date': '29-Dec-17',
    'High': 135.0,
    'Low': 132.0,
    'Open': 134.0,
    'StockCode': 'smmt',
    'Volume': 924700.0},
   {'Close': 135.0,
    'Date': '28-Dec-17',
    'High': 137.0,
    'Low': 132.0,
    'Open': 133.0,
    'StockCode': 'smmt',
    'Volume': 775200.0},
   {'Close': 133.0,
    'Date': '27-Dec-17',
    'High': 133.0,
    'Low': 132.0,
    'Open': 133.0,
    'StockCode': 'smmt',
    'Volume': 78600.0},
   {'Close': 134.0,
    'Date': '22-Dec-17',
    'High': 135.0,
    'Low': 132.0,
    'Open': 134.0,
    'StockCode': 'smmt',
    'Volume': 395100.0},
   {'Close': 133.0,
    'Date': '21-Dec-17',
    'High': 133.0,
    'Low': 132.0,
    'Open': 133.0,
    'StockCode': 'smmt',
    'Volume': 626200.0},
   {'Close': 133.0,
    'Date': '20-Dec-17',
    'High': 134.0,
    'Low': 132.0,
    'Open': 132.0,
    'StockCode': 'smmt',
    'Volume': 658800.0},
   {'Close': 133.0,
    'Date': '19-Dec-17',
    'High': 135.0,
    'Low': 133.0,
   

In [ ]:
def transform_to_percentage(daily_price_list):
    stack_result = []
    for i in range(0, len(daily_price_list) - 1) :
        curr_price = daily_price_list[i]
        prev_price = daily_price_list[i + 1]
        new_curr_price = {key : (curr_price[key] - prev_price[key] ) / prev_price[key] \
                         for key in curr_price.keys()  \
                         if not any(x in key for x in ['Date', 'Code']) and prev_price[key] > 0}
        stack_result.append(new_curr_price)
    return stack_result

df_3 = df_2.map(lambda x: (x[0], transform_to_percentage(x[1])))

In [62]:
df_3.take(1)

[('smmt',
  [{'Close': -0.014814814814814815,
    'High': -0.014598540145985401,
    'Low': 0.0,
    'Open': 0.007518796992481203,
    'Volume': 0.19285345717234262},
   {'Close': 0.015037593984962405,
    'High': 0.03007518796992481,
    'Low': 0.0,
    'Open': 0.0,
    'Volume': 8.86259541984733},
   {'Close': -0.007462686567164179,
    'High': -0.014814814814814815,
    'Low': 0.0,
    'Open': -0.007462686567164179,
    'Volume': -0.8010630220197419},
   {'Close': 0.007518796992481203,
    'High': 0.015037593984962405,
    'Low': 0.0,
    'Open': 0.007518796992481203,
    'Volume': -0.36905142127115936},
   {'Close': 0.0,
    'High': -0.007462686567164179,
    'Low': 0.0,
    'Open': 0.007575757575757576,
    'Volume': -0.04948391013964785},
   {'Close': 0.0,
    'High': -0.007407407407407408,
    'Low': -0.007518796992481203,
    'Open': -0.014925373134328358,
    'Volume': 3.054153846153846},
   {'Close': -0.007462686567164179,
    'High': 0.007462686567164179,
    'Low': 0.0,
   

In [ ]:
def stack_feature(daily_price_list, days_to_be_stack):
    stack_result = []
    for i in range(0, len(daily_price_list)) :
        block_size = i + days_to_be_stack
        if block_size <= len(daily_price_list) :
            stack_result.append(daily_price_list[i:block_size])
    return stack_result

df_4 = df_3.map(lambda x: (x[0],stack_feature(x[1], days_to_be_stacked)))

In [ ]:
def generate_target_feature(code, daily_price_seq):
    target = []
    features = []
    for sequence in daily_price_seq :
        target_feature = generate_target_feature_per_sequence(sequence)
        target.append(target_feature['target'])
        features.append(target_feature['features'])
    
    return {'target' : target, 'feature' : features}

def generate_target_feature_per_sequence(x):
    target = x[0]
    feature_list = {}
    src_feature_list = x[1:]
    for (daily_price, i) in zip(src_feature_list, range(0, len(src_feature_list))):
        for key in daily_price.keys():
            feature_list[key+'_'+str(i)] = daily_price[key]
    return {'target' : target, 'features' : feature_list}

In [ ]:
df_5 = df_4.map(lambda x: (x[0], generate_target_feature(x[0], x[1])))

In [ ]:
def filter_label_feature(x):
    code = x[0]
    data = x[1]
    target = data['target']
    target_list = [t['High'] for t in target]
    src_feature_list = data['feature']
    feature_list = []
    
    for f in src_feature_list:
        feature = [f[key] for key in f.keys() if not any(x in key for x in ['Date', 'Code'])]
        
        # 5 for High, Low, Open, CLose and Volume
        if len(feature) == (days_to_be_stacked-1) * 5  :
            feature_list.append(feature)
        else :
            print(code + str(data))
        
    return {'code': code, 'label' : target_list, 'features' : feature_list}

In [ ]:
df_6 = df_5.map(lambda x: filter_label_feature(x))

In [ ]:
df_7 = df_6.flatMap(lambda x :zip( x['features'],x['label'])) \
        .toDF(["features", "label"])

In [ ]:
df_7.show()

In [ ]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
final_data = df_7.select(
    df_7["label"],
    list_to_vector_udf(df_7["features"]).alias("features")
)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Load and parse the data file, converting it to a DataFrame.
# data = spark.read.format("libsvm").load("sample_libsvm_data.txt")
data = final_data

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.rdd.randomSplit([0.7, 0.3])
trainingData = trainingData.toDF()
testData = testData.toDF()

# Train a GBT model.
gbt = GBTRegressor(maxDepth=15, maxBins=32, \
                   minInstancesPerNode=1, minInfoGain=0.0, maxMemoryInMB=1024, \
                   cacheNodeIds=False, checkpointInterval=10, \
                   maxIter=20, stepSize=0.1, seed=None, subsamplingRate=1.0)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

gbtModel = model.stages[1]
print(gbtModel)  # summary only

In [ ]:
predictions = model.transform(trainingData)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

gbtModel = model.stages[1]
print(gbtModel)  # summary only

In [43]:
model.save('gbtModel_30_features_f')

In [44]:
savedModel = model.load('gbtModel_final')

In [45]:
# Make predictions.
predictions = savedModel.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

+--------------------+--------------------+--------------------+
|          prediction|               label|            features|
+--------------------+--------------------+--------------------+
|0.008657251974849739|0.007462686567164179|[-0.0145985401459...|
|0.007563331736679668|-0.00740740740740...|[0.01481481481481...|
|0.005279755386048...|0.014814814814814815|[0.0,0.0075757575...|
|0.011577714041060755|                 0.0|[0.0,-0.022222222...|
| 0.01328275086322394|0.014492753623188406|[0.00751879699248...|
+--------------------+--------------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.0659812


In [ ]:
days = 3

def convert_float( x):
#     code = x['filename'].split('/')[5][:4]
    try:
        new_x = {'Close' : float(x['Close']), \
#                  'StockCode' : code,\
                 'Date' : x['Date'], \
                 'High' : float(x['High']), \
                 'Low' : float(x['Low']), \
                 'Open' : float(x['Open'])}
        return new_x
    except:
        new_x = {'Close' : float(x['Close']), \
#                  'StockCode' : code,\
                 'Date' : x['Date'], \
                 'High' : float(x['Close']), \
                 'Low' : float(x['Close']), \
                 'Open' : float(x['Close'])}
        return new_x

In [ ]:
X_data = []
y_data = []
result = {}
df_float = {}
file_dir = "../stock-predictions/until31_2017/"
for fname in os.listdir(file_dir):
    df = spark.read.format("csv").option("header", "true").load(file_dir + fname).limit(days)
    df = df.withColumn("id", monotonically_increasing_id())
    df_float = df.rdd.map(convert_float).toDF()
    feature_list = df_float.rdd.collect()
    target = feature_list[-1:][0] #last feature will be target
    feature = feature_list[:-1]
    feature = np.reshape(feature, (len(df_float.columns)*len(feature)))
    X_data.append(feature)
    y_data.append(target['High'])
    
    result['feature'] = feature
    result['target'] = target['High']
    break

In [ ]:
banyak_hari_yang_mau_distack=4
def datediubahkebanyakhari(timestamp,banyak_hari_yang_mau_distack=4):
    timestamp = time.mktime(datetime.datetime.strptime(timestamp,'%d-%b-%y').timetuple())
    return int((timestamp/(60*60*24*banyak_hari_yang_mau_distack)))

In [ ]:
def bikin_lurus(x):
    return_dict = {}
    for (data,i) in zip(x,range(0,len(x))):
        for key in data.keys():
            return_dict[key+'_'+str(i)] = data[key]
    return return_dict

In [ ]:
coba_data = data_rdd.map(lambda x: ( (x['StockCode'], datediubahkebanyakhari(x['Date'])) , x)) \
            .groupByKey().mapValues(list) \
            .map(lambda x: (x[0], bikin_lurus(x[1])))

In [ ]:
coba_data.take(5)